In [6]:
import pandas as pd
import psycopg2

In [7]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [8]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

6. Предметный 
✍ Переходим к самому интересному — нам предстоит исследовать требования работодателей к дата-сайентистам.

Задание 6.1
Сколько вакансий имеет отношение к данным

In [8]:
query = f''' select count(2)

 from public.vacancies v
    where 
      name ilike '%данн%'
      or name ilike '%data%' 
          
       '''  

In [9]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_1308\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,1771


Задание 6.2
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
‘data scientist’;
‘data science’;
‘исследователь данных’;
‘ML’ (здесь не нужно брать вакансии по HTML);
‘machine learning’;
‘машинн%обучен%’.

Считаем вакансиями для специалистов уровня Junior следующие:
в названии есть слово “junior” или
требуемый опыт — «Нет опыта» или
тип трудоустройства — «Стажировка».

In [31]:
query = f''' select count(*)
     from public.vacancies v
     where 
     (name ilike '%data scientist%'
     or name ilike '%data science%'
     or name ilike '%исследователь данных%'
     or (name ilike '%ML%' 
     and name not ilike '%HTML%')
     or name ilike '%machine learning%'
     or name ilike 'машинн%обучен%')
     and (name ilike '%junior%'
     or experience ilike '%Нет опыта%'
     or employment ilike '%Стажировка%')
    '''  

In [33]:
wquery = '''
SELECT COUNT(*) AS junior_ds_count
FROM public.vacancies
WHERE 
    (name ILIKE '%data scientist%'
    OR name ILIKE '%data science%'
    OR name ILIKE '%исследователь данных%'
    OR (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%')
    OR name ILIKE '%machine learning%'
    OR name ILIKE 'машинн%обучен%')
    AND 
    (name ILIKE '%junior%'
    OR experience ILIKE '%Нет опыта%'
    OR employment ILIKE '%Стажировка%')
'''

In [34]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,49


qSELECT COUNT(*) AS junior_ds_vacancies
FROM vacancies
WHERE (
    -- Условия по названию (Data Science / ML)
    LOWER(title) LIKE '%data scientist%'
    OR LOWER(title) LIKE '%data science%'
    OR LOWER(title) LIKE '%исследователь данных%'
    OR (LOWER(title) LIKE '%ml%' AND LOWER(title) NOT LIKE '%html%')
    OR LOWER(title) LIKE '%machine learning%'
    OR LOWER(title) LIKE '%машинн%обучен%'
)
AND (
    -- Условия для Junior
    LOWER(title) LIKE '%junior%'
    OR experience = 'Нет опыта'
    OR employment = 'Стажировка'
);

Чтобы определить количество подходящих вакансий для начинающего дата-сайентиста, нужно выполнить следующие шаги:

1. Фильтрация вакансий по названию (Data Science / ML)
Отбираем вакансии, в названии которых есть хотя бы одно из указанных сочетаний:

data scientist

data science

исследователь данных

ML (но не HTML)

machine learning

машинн%обучен% (здесь % означает любые символы, например, "машинное обучение", "машинному обучению" и т. д.)

2. Фильтрация по уровню Junior
Из отобранных вакансий выбираем те, которые подходят для начинающих специалистов, по критериям:

В названии есть слово junior (регистр не учитывается, например, Junior, JUNIOR, junior), или

Требуемый опыт работы — Нет опыта, или

Тип трудоустройства — Стажировка.

Итоговый запрос (пример для SQL)
sql
Copy
SELECT COUNT(*) AS junior_ds_vacancies
FROM vacancies
WHERE (
    -- Условия по названию (Data Science / ML)
    LOWER(title) LIKE '%data scientist%'
    OR LOWER(title) LIKE '%data science%'
    OR LOWER(title) LIKE '%исследователь данных%'
    OR (LOWER(title) LIKE '%ml%' AND LOWER(title) NOT LIKE '%html%')
    OR LOWER(title) LIKE '%machine learning%'
    OR LOWER(title) LIKE '%машинн%обучен%'
)
AND (
    -- Условия для Junior
    LOWER(title) LIKE '%junior%'
    OR experience = 'Нет опыта'
    OR employment = 'Стажировка'
);
Приблизительное количество вакансий (оценка)
Точное число зависит от базы данных (например, HeadHunter, LinkedIn, Indeed). В среднем, для российского рынка (на HH.ru) может быть от 50 до 200+ Junior-вакансий в Data Science / ML, но это сильно зависит от времени года и текущего спроса.

Если у вас есть доступ к конкретной базе данных, можно выполнить запрос выше для точного подсчёта.

New chat
Message DeepSeek


Задание 6.3
Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
В качестве ответа запишите количество, которое вернул запрос.

In [42]:
query = f''' select count(*)
     from public.vacancies v
     where
     (name ilike '%data scientist%'
     or name ilike '%data science%'
     or name ilike '%исследователь данных%'
     or (name ilike '%ML%' and name not ilike '%HTML%')
     or name ilike '%machine learning%'
     or name ilike '%машинн%обучен%')
     
     and (key_skills ilike '%SQL%' 
     or key_skills ilike '%postgres%')
         
    '''  


In [21]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740
...,...,...,...,...,...,...,...,...,...,...
49192,55589514,Специалист по информационным технологиям,Электронный документооборот\t1С: Предприятие 8...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,66,5109332
49193,55029668,Руководитель процессного офиса,Оптимизация бизнес-процессов\tСтратегическое м...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,26,51333
49194,66172263,Начальник сервисной службы,Организаторские навыки\tУправление персоналом\...,Полный день,От 1 года до 3 лет,Полная занятость,60000.0,NaN,1930,4094700
49195,66180086,Senior IOS разработчик,Swift\tRealm\tiOS\tАнглийский язык\tMVP\tMVVM\...,Полный день,От 1 года до 3 лет,Полная занятость,146456.0,244094.0,160,4469939


In [43]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,229


Задание 6.4
С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
В качестве ответа запишите количество, которое вернул запрос.

In [46]:
query = f''' select count(*)
     from public.vacancies v
     where
     (name ilike '%data scientist%'
     or name ilike '%data science%'
     or name ilike '%исследователь данных%'
     or (name ilike '%ML%' and name not ilike '%HTML%')
     or name ilike '%machine learning%'
     or name ilike '%машинн%обучен%')
     
     and key_skills ilike '%Python%' 
    
         
    '''  


In [47]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,357


In [72]:
query = f''' select name, key_skills
     from public.vacancies v
     where
     name ilike '%DS%'

  
         
    '''  

In [82]:
query = '''
SELECT ROUND(AVG(skill_count), 2) AS avg_skills
FROM (
    SELECT 
        CASE 
            WHEN key_skills IS NULL OR TRIM(key_skills) = '' THEN 0
            ELSE ARRAY_LENGTH(STRING_TO_ARRAY(key_skills, ','), 1)
        END AS skill_count
    FROM public.vacancies
    WHERE 
        name ~* 'data scientist|data science|исследователь данных|machine learning|машинн.*обучен'
        OR (name ~* 'ML' AND name !~* 'HTML')
) t;
'''

In [86]:
query = '''
SELECT ROUND(AVG(
    CASE 
        WHEN key_skills IS NULL OR key_skills = '' THEN 0
        ELSE (LENGTH(key_skills) - LENGTH(REPLACE(key_skills, ',', ''))) + 1
    END
), 2) AS avg_skills
FROM public.vacancies
WHERE 
    name ~* 'data scientist|data science|исследователь данных|machine learning|машинн.*обучен'
    OR (name ~* 'ML' AND name !~* 'HTML');
'''

In [87]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,avg_skills
0,0.98


Подсказка (1 из 3): Посчитать количество определённых символов в строке с помощью SQL можно следующим образом: из длины исходной строки вычесть длину строки, в которой все вхождения этого символа заменены на пустую строку (''). Работа со строками изучалась в юните «*BONUS. SQL-5. Типы данных 6. Функции и операторы для работы со строками».

In [182]:
query = f''' select name, REPLACE(key_skills, ' ', ',')
     from public.vacancies v
      where
     (name ilike '%data scientist%'
     or name ilike '%data science%'
     or name ilike '%исследователь данных%'
     or (name ilike '%ML%' and name not ilike '%HTML%')
     or name ilike '%machine learning%'
     or name ilike '%машинн%обучен%')
     
     and (key_skills ilike '%SQL%' 
     or key_skills ilike '%postgres%')
         
    '''  


In [196]:
 query = f''' select name, v.key_skills
      from public.vacancies v
      where
      name ~* 'data scientist|data science|исследователь данных|machine learning|машинн%обучен' 
      or (name ~* 'ML' and name !~* 'HTML')
         
    '''  


In [198]:
query = f''' select name, REPLACE(key_skills, ' ', ',')
     from public.vacancies v
     where
      name ~* 'data scientist|data science|исследователь данных|machine learning|машинн%обучен' 
      or (name ~* 'ML' and name !~* 'HTML')
         
    '''  


In [199]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,replace
0,Junior+ CV/ML developer,"Python\tSQL\tРабота,в,команде\tOpenCV\tML"
1,Data Scientist,"Oracle,Pl/SQL\tБазы,данных\tMachine,Learning\t..."
2,Data Scientist (Junior+),"Python\tOpenCV\tTensorflow\tComputer,Vision\tН..."
3,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit
4,QA Automation TeamLead,"Atlassian,Jira\tФункциональное,тестирование\tS..."
...,...,...
519,C++ разработчик (QML),"Qt\tООП\tC++\tMS,Visual,Studio\tРазработка,ком..."
520,Middle+/Senior Data Scientist,"SCALA\tSQL\tМатематический,анализ\tСтатистичес..."
521,ML Engineer Team Lead (to Cyprus),"Python\tLinux\tKubernetes\tAWS\tGit\tPeople,Ma..."
522,Системный аналитик /Middle+/Senior/TeamLead,"UML\tSQL\tBPMN\tSOAP\tREST\tПостановка,задач,р..."


In [117]:
query = f''' select  length(replace(key_skills,',',
''))
     from public.vacancies v
     
         
    '''  


In [118]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,length
0,138.0
1,125.0
2,43.0
3,43.0
4,NaN
...,...
49192,276.0
49193,77.0
49194,104.0
49195,52.0


In [170]:
query = f''' select name, key_skills
     from public.vacancies v
     where name ilike '%DS%'
     '''

In [174]:
query = f''' select name, REPLACE(key_skills, ' ', ',')
     from public.vacancies v
     where name ilike '%DS%'
     '''

In [175]:
df = pd.read_sql_query(query, connection)
df

C:\Users\Станислав\AppData\Local\Temp\ipykernel_6684\137330334.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,replace
0,Консультант SAP PPDS,"Разработка,инструкций\tSAP,ERP\tSAP,PPDS"
1,DevOps Engineer with clouds (Poland),"Python\tAnsible\tJenkins\tLinux\tАнглийский,яз..."
2,"GDS Expert, Flight Booking Platform","Java\tАнглийский,язык\tBusiness,Development\tП..."
3,"GDS Expert, Flight Booking Platform","Java\tАнглийский,язык\tBusiness,Development\tП..."
4,"GDS Expert, Flight Booking Platform","Java\tАнглийский,язык\tBusiness,Development\tП..."
5,Cтажер по ведению рекламы в Google Ads/Faceboo...,"Работа,с,большим,объемом,информации\tКонтекстн..."
6,IDS Infrastructure and Technology Supervisor,None
7,Ведущий программист DS/DE,"Python\tSQL\tGit\tBig,Data\tSpark"
8,Ведущий программист DS/DE,"Python\tSQL\tGit\tBig,Data\tSpark"
9,Ведущий программист DS/DE,"Python\tSQL\tGit\tBig,Data\tSpark"
